# Merge Airport and Weather Data
    domestic ontime performance data. 
    NOAA hourly time data.
    merge based on the airport code and closest weather observation time point.
    

In [24]:
import numpy as np
import pandas as pd

In [25]:
wban_code = {} # wban= weather station code

In [26]:
years=['2014','2015']
months =['4','5','6','7','8','9','10','11','12','1', '2','3']
# months =['1','2']
path ='F:\\Dropbox\\Flight\\Data\\'

# need to looping.
# select big airport
# data have 304 airport.


## 1. Make WBAN map code 
    dictionary: wban_code= {airport:code}

In [27]:
# This WBANmap function reads the station code file and pick WBAN and its number to create the wban map

def WBANmap(path, year):
    wban_code={}
    
    for i in range(1,13):
        mon= str(i)
        if(i < 10):
            mon = '0' + mon
        mapfile_name =path+'Weather\\'+year +mon+'station.txt'

        try:        
            map_file = open(mapfile_name) #open file
            print(mapfile_name)
        except FileNotFoundError:
            continue

        lines = map_file.readlines()
        for i in range(1, len(lines)):

            items = lines[i].split('|') #parse the cell
            wban_code[items[2]] = items[0] #
        map_file.close()    
            # set dictionary 
            #item[2] is CallSign: items[0] is WBAN : CallSign is enclude the airport code.
    # print(lines[0])
    # print(lines[5]) 
    #creat map file
    mpfile = open('mapfile_'+year+'.txt', 'w')
    mpfile.write(str(wban_code))
    mpfile.close()  
    return(wban_code)
    print('map setted')

## 2. Read Airport data 
    
### 2.0 It can give a rule to select specific airports based on the number of flights per month.
        : ex) df = df.loc[df.index[Nflights_orig>=500]
### 2.1 Prepare the time variable for matching with weather data.

### 2.2 Mapping a WBAN code to the airport data.
    
      
    Both departure data +WBAN code
         Arrival data +WBAN code
    Remove the empty variables.

In [28]:
def call_airport(path,year,month):
    filename= path+ '\\Data_ontime\\On_Time_On_Time_Performance_'+year+'_'+month+'.csv'
    df= pd.read_csv(filename)
    print('read'+filename)
#     print(df.head())
    # Find all unique airports
    all_airports, airport_inverse, airport_count = np.unique(df['Origin'],return_counts=True,return_inverse=True)
    # print(all_airports)
    # print(len(all_airports))
    Nflights_orig = np.zeros(len(airport_inverse))
    for i in range(len(all_airports)):
        Nflights_orig[np.where(airport_inverse==i)] = airport_count[i]
    #print(Nflights_orig)
    # df = df.loc[df.index[Nflights_orig>=500]]
    keyval=['Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'FlightDate',
           'UniqueCarrier', 'AirlineID', 'Carrier', 'TailNum',
            'FlightNum', 'Origin', 'OriginCityName', 'OriginState', 'OriginStateFips',
            'Dest', 'DestCityName', 'DestState', 'DestStateFips', 
            'CRSDepTime', 'DepTime', 'DepDelay', 'DepDelayMinutes', 'DepDel15',
            'DepartureDelayGroups', 'DepTimeBlk',
            'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn',
            'CRSArrTime', 'ArrTime', 'ArrDelay', 'ArrDelayMinutes', 'ArrDel15',
            'ArrivalDelayGroups', 'ArrTimeBlk', 'Cancelled', 'CancellationCode',
            'Diverted', 'CRSElapsedTime', 'ActualElapsedTime', 
            'AirTime', 'Flights', 'Distance', 'DistanceGroup',
            'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay',
            'FirstDepTime', 'TotalAddGTime', 'LongestAddGTime', 'DivAirportLandings',
            'DivReachedDest','DivActualElapsedTime', 'DivArrDelay', 'DivDistance',
            'Div1Airport', 'Div1AirportID', 'Div1AirportSeqID', 'Div1WheelsOn']
    
    df = df[keyval]
# set a new key for future merge.    
    df['key']= range(1,(len(df)+1),1) 
    
#     print(df['FlightDate'].head(2))
#     print(df['CRSDepTime'].tail())
     # data look like '1' .  need to change format like '0001'.
     # map({:04}.format) specifies convert the input digit in 4 chars by prefixing '0'.
    df['sd_time'] = df['CRSDepTime'].map("{:04}".format)
    
    df['sa_time'] = df['CRSArrTime'].map("{:04}".format)
    
#change the wrong formatted time : since to datetime can read the 24:00  so need to change 00:00
    df['sa_time'] = df['sa_time'].str.replace('2400','0000')
    
    # split the hours and minuate
    # For departure
    times = pd.to_datetime(df['sd_time'],format= '%H%M').dt.hour
    mins= pd.to_datetime(df['sd_time'],format= '%H%M' ).dt.minute
    # Convert exact time format.
    df['S_Dep_time'] = pd.DataFrame({'time': pd.to_datetime(df['FlightDate'])+pd.to_timedelta(times,unit='h')+pd.to_timedelta(mins,unit='m')})
    
#     print(df['S_Dep_time'].head())
    
    # For Arrival 
    times = pd.to_datetime(df['sa_time'],format= '%H%M').dt.hour
    mins= pd.to_datetime(df['sa_time'],format= '%H%M' ).dt.minute
    # Convert exact time format.
    df['S_Arr_time'] = pd.DataFrame({'time': pd.to_datetime(df['FlightDate'])+pd.to_timedelta(times,unit='h')+pd.to_timedelta(mins,unit='m')})

#     print(df['S_Arr_time'].head())

    # Since wban_code is dictionary, when we put the key number like wban_code['DCA'] then it returns the coresponding value.

    df['Dep_wban'] = df['Origin'].map(wban_code)
    df['Arr_wban'] = df['Dest'].map(wban_code)
    
    return(df)


## 3. Read weather data

In [29]:
def call_weather(path,year,month):
    i= int(month)
    if i<10:
        month='0'+month
    hourly_file_name = path+'Weather\\'+year+ month+'hourly.txt'
    df_w= pd.read_csv(hourly_file_name,dtype={'WBAN': object, 'Time': object})
    print('read'+hourly_file_name)
    Date= pd.to_datetime(df_w.Date,format='%Y%m%d') 
    
    times=  pd.to_datetime(df_w['Time'],format= '%H%M' ).dt.hour
    mins= pd.to_datetime(df_w['Time'],format= '%H%M' ).dt.minute
    df_w['time']= pd.DataFrame({'time': pd.to_datetime(Date)+pd.to_timedelta(times,unit='h')+pd.to_timedelta(mins,unit='m')})
#     df_w.time.head(5)
    return (df_w)

In [30]:
## Daily weather

def call_daily(path,year,month):
    i= int(month)
    if i<10:
        month='0'+month
    daily_file_name = path+'Weather\\'+year+ month+'daily.txt'
    df_day= pd.read_csv(daily_file_name,dtype={'WBAN': object, 'Datetime': object})
    print('read'+daily_file_name)
    return(df_day)    

In [31]:
df_day = call_daily(path,'2014','4')
df_day.head()


readF:\Dropbox\Flight\Data\Weather\201404daily.txt


,WBAN,YearMonthDay,Tmax,TmaxFlag,Tmin,TminFlag,Tavg,TavgFlag,Depart,DepartFlag,...,AvgSpeed,AvgSpeedFlag,Max5Speed,Max5SpeedFlag,Max5Dir,Max5DirFlag,Max2Speed,Max2SpeedFlag,Max2Dir,Max2DirFlag
0,03013,20140401,72,,25,,49,,M,,...,18.6,,43,,130,,33,,110,
1,03013,20140402,68,,32,,50,,M,,...,14.7,,33,,070,,29,,070,
2,03013,20140403,45,,32,,39,,M,,...,11.5,,31,,360,,24,,350,
3,03013,20140404,62,,27,,45,,M,,...,7.6,,22,,170,,17,,170,
4,03013,20140405,61,,28,,45,,M,,...,9.4,,40,,190,,33,,200,


In [33]:
keys = ['WBAN', 'YearMonthDay','Tmax','Tmin','Water1','SnowFall','PrecipTotal']
df_day= df_day[keys]

## 4. Merge.

In [59]:
pd.options.mode.chained_assignment = None  # default='warn'

In [60]:
def merge(path, year, month):
    # call data
    
    df= call_airport(path,year,month)
    df_w=call_weather(path,year,month)
    
    # select the specific weather data which will be used.
    airports = ['BOS','PVD','MHT','ORH','LAX', 'SNA', 'BUR', 'LGB']
#     airports = ['BUR','LGB']
    # airports=['ABQ', 'ALB', 'AMA', 'ANC', 'ATL', 'AUS', 'BDL', 'BHM', 'BNA',
    #        'BOI', 'BOS', 'BTR', 'BUF', 'BUR', 'BWI', 'CAE', 'CHS', 'CLE',
    #        'CLT', 'CMH', 'COS', 'CVG', 'DAL', 'DAY', 'DCA', 'DEN', 'DFW',
    #        'DSM', 'DTW', 'ELP', 'EWR', 'FAT', 'FLL', 'GEG', 'GRR', 'GSO',
    #        'GSP', 'HNL', 'HOU', 'HPN', 'IAD', 'IAH', 'ICT', 'IND', 'JAN',
    #        'JAX', 'JFK', 'KOA', 'LAS', 'LAX', 'LGA', 'LGB', 'LIH', 'LIT',
    #        'MAF', 'MCI', 'MCO', 'MDW', 'MEM', 'MIA', 'MKE', 'MSN', 'MSP',
    #        'MSY', 'OAK', 'OGG', 'OKC', 'OMA', 'ONT', 'ORD', 'ORF', 'PBI',
    #        'PDX', 'PHL', 'PHX', 'PIT', 'PNS', 'PSP', 'PVD', 'RDU', 'RIC',
    #        'RNO', 'ROC', 'RSW', 'SAN', 'SAT', 'SAV', 'SBA', 'SDF', 'SEA',
    #        'SFO', 'SJC', 'SJU', 'SLC', 'SMF', 'SNA', 'STL', 'TPA', 'TUL',
    #        'TUS', 'TYS', 'XNA']
    condition=['Dep','Arr']
    
    dfList= []

    for airport in airports:
        for CON in condition:
            code=wban_code[airport]
            print(year +month)
            print(airport + ' WBAN_code :'+code) 

            df1= df.loc[df[CON+'_wban']==code]  # seperatly add the weather data for arrival and departure.
            df1['con']=CON
            obs= str(len(df1))
            print('# of obs:'+ obs)
            df1['time']= df1['S_'+CON+'_time']
                
            df2= df_w.loc[df_w['WBAN']==code]
            print('length of weather data_'+str(len(df2)))
            df2['next_obs']= df2.time.shift(-1)
            df2['obs1']= pd.to_timedelta(df2['time'])
            df2['obs2']= pd.to_timedelta(df2['next_obs'])
            df2['mid'] = df2['next_obs']-(df2['obs2']-df2['obs1'])/2
            df2['mid'].fillna(method= 'ffill',inplace=True)
            df2['next_obs'].fillna(method= 'ffill',inplace=True)
            df2['id']= range(1,(len(df2)+1),1)
            del df2['obs1']
            del df2['obs2']

            airport_time= df1[['time','key']]
            weather_time= df2[['time','id','mid']]
            whole_time= airport_time.append(weather_time, ignore_index=True)

            #reset index
            whole_time.index= range(1,len(whole_time)+1)
            whole_time= whole_time.sort_values('time')
            whole_time['new_id'] = whole_time['id']
            whole_time['mid_id'] = whole_time['id']
            whole_time['mid'].fillna(method= 'ffill', inplace=True)
            whole_time.iat[-1,2] =whole_time.iat[-1,3] # replace last midtime as real time
            whole_time['mid_id'].fillna(method= 'ffill', inplace = True)

            whole_time['new_id']= np.where((( whole_time['time'] < whole_time['mid'] )),whole_time['mid_id'],whole_time['mid_id']+1)

            #remove_weather_time
            whole_time= whole_time[whole_time['id'].isnull()==True]
            # merge with new_id data
            df3= pd.merge(df1,whole_time, on ='key', indicator =False)
            print(len(df3))    
            del df3['id']
            del df3['mid']
            del df3['mid_id']
            del df3['time_x']
            del df3['time_y']
#             df3 = df2.drop(['id', 'mid', 'mid_id', 'time_x','time_y'], axis = 1) 
# axis =1 denotes that we are referring to a column, no a row.
# results are same as above del 
#             df3.to_csv('df3.csv')
            # finalize the merge : matching with weather data.
            result = pd.merge(df3,df2, how='left', left_on='new_id', right_on= 'id',indicator=True)
            print("merged data_ "+ str(len(result)))
            
            del result['mid']
            del result['next_obs']
            
            dfList.append(result)
    appendDf= pd.concat(dfList, axis=0)   
    appendDf.to_csv(path+'results_'+year+month+'.csv')
    

## 5.Execute the merge.

In [61]:
# this loop call wban_code function, and call merge function. it will save the merged data at setted folder.

for year in years:
    print(year)
    wban_code= WBANmap(path,year)
    for month in months:
        print(month)
        try:
            merge(path,year,month)
        except FileNotFoundError:
            continue
        

2014
F:\Dropbox\Flight\Data\Weather\201404station.txt
F:\Dropbox\Flight\Data\Weather\201405station.txt
F:\Dropbox\Flight\Data\Weather\201406station.txt
F:\Dropbox\Flight\Data\Weather\201407station.txt
F:\Dropbox\Flight\Data\Weather\201408station.txt
F:\Dropbox\Flight\Data\Weather\201409station.txt
F:\Dropbox\Flight\Data\Weather\201410station.txt
F:\Dropbox\Flight\Data\Weather\201411station.txt
F:\Dropbox\Flight\Data\Weather\201412station.txt
4


C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (48,69,76,77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


readF:\Dropbox\Flight\Data\\Data_ontime\On_Time_On_Time_Performance_2014_4.csv


C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (10,12,18,20,24,26,42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


readF:\Dropbox\Flight\Data\Weather\201404hourly.txt
20144
BOS WBAN_code :14739
# of obs:9373
length of weather data_857
9373
merged data_ 9373
20144
BOS WBAN_code :14739
# of obs:9374
length of weather data_857
9374
merged data_ 9374
20144
PVD WBAN_code :14765
# of obs:978
length of weather data_887
978
merged data_ 978
20144
PVD WBAN_code :14765
# of obs:975
length of weather data_887
975
merged data_ 975
20144
MHT WBAN_code :14710
# of obs:532
length of weather data_832
532
merged data_ 532
20144
MHT WBAN_code :14710
# of obs:533
length of weather data_832
533
merged data_ 533
20144
ORH WBAN_code :94746
# of obs:60
length of weather data_985
60
merged data_ 60
20144
ORH WBAN_code :94746
# of obs:60
length of weather data_985
60
merged data_ 60
20144
LAX WBAN_code :23174
# of obs:18544
length of weather data_822
18544
merged data_ 18544
20144
LAX WBAN_code :23174
# of obs:18549
length of weather data_822
18549
merged data_ 18549
20144
SNA WBAN_code :93184
# of obs:3215
length of weath

...

readF:\Dropbox\Flight\Data\Weather\201409hourly.txt
20149
BOS WBAN_code :14739
# of obs:9106
length of weather data_793
9106
merged data_ 9106
20149
BOS WBAN_code :14739
# of obs:9105
length of weather data_793
9105
merged data_ 9105
20149
PVD WBAN_code :14765
# of obs:1036
length of weather data_823
1036
merged data_ 1036
20149
PVD WBAN_code :14765
# of obs:1036
length of weather data_823
1036
merged data_ 1036
20149
MHT WBAN_code :14710
# of obs:653
length of weather data_857
653
merged data_ 653
20149
MHT WBAN_code :14710
# of obs:655
length of weather data_857
655
merged data_ 655
20149
ORH WBAN_code :94746
# of obs:60
length of weather data_1015
60
merged data_ 60
20149
ORH WBAN_code :94746
# of obs:60
length of weather data_1015
60
merged data_ 60
20149
LAX WBAN_code :23174
# of obs:17589
length of weather data_792
17589
merged data_ 17589
20149
LAX WBAN_code :23174
# of obs:17585
length of weather data_792
17585
merged data_ 17585
20149
SNA WBAN_code :93184
# of obs:3243
length 

C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (77,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


readF:\Dropbox\Flight\Data\\Data_ontime\On_Time_On_Time_Performance_2015_1.csv


C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2881: DtypeWarning: Columns (10,12,18,20,24,26,30,42) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


readF:\Dropbox\Flight\Data\Weather\201501hourly.txt
20151
BOS WBAN_code :14739
# of obs:8837
length of weather data_953
8837
merged data_ 8837
20151
BOS WBAN_code :14739
# of obs:8841
length of weather data_953
8841
merged data_ 8841
20151
PVD WBAN_code :14765
# of obs:1024
length of weather data_992
1024
merged data_ 1024
20151
PVD WBAN_code :14765
# of obs:1024
length of weather data_992
1024
merged data_ 1024
20151
MHT WBAN_code :14710
# of obs:469
length of weather data_974
469
merged data_ 469
20151
MHT WBAN_code :14710
# of obs:471
length of weather data_974
471
merged data_ 471
20151
ORH WBAN_code :94746
# of obs:62
length of weather data_1109
62
merged data_ 62
20151
ORH WBAN_code :94746
# of obs:62
length of weather data_1109
62
merged data_ 62
20151
LAX WBAN_code :23174
# of obs:17340
length of weather data_848
17340
merged data_ 17340
20151
LAX WBAN_code :23174
# of obs:17332
length of weather data_848
17332
merged data_ 17332
20151
SNA WBAN_code :93184
# of obs:3185
length 

## 6. Appending Above files

In [62]:
import os
import glob

def appender(indir= "F:/Dropbox/Flight/Data/", output = 'F:/Dropbox/Flight/Data/cleaned_data/prepared_data.csv'):
    os.chdir(indir)
    fileList= glob.glob("results_20*.csv")
    
    #creating empty list
    dfList= []
    print(fileList)
    # read the file and put it to the list.
    
    for filename in fileList:
        print(filename)
        df= pd.read_csv(filename) 
        dfList.append(df)
    ## append each row to one dataframe.    
    
    appendDf= pd.concat(dfList, axis=0)
    appendDf.to_csv(output)
    
appender()


['results_201410.csv', 'results_201411.csv', 'results_201412.csv', 'results_20144.csv', 'results_20145.csv', 'results_20146.csv', 'results_20147.csv', 'results_20148.csv', 'results_20149.csv', 'results_20151.csv', 'results_20152.csv', 'results_20153.csv']
results_201410.csv


C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2827: DtypeWarning: Columns (87,89,91,93,95,97,103) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


results_201411.csv


C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2827: DtypeWarning: Columns (79,83,85,87,89,91,93,95,97,103,115) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


results_201412.csv


...

C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2827: DtypeWarning: Columns (83,85,87,89,91,93,95,97,103) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


results_20151.csv
results_20152.csv


C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2827: DtypeWarning: Columns (83,85,87,89,91,93,95,97,103,115) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


results_20153.csv


C:\Users\Jukwan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2827: DtypeWarning: Columns (87,89,91,93,95,97) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
